In [53]:
!pip install pytube
!pip install unidecode

In [54]:
import os
import numpy as np
import pandas as pd
from pytube import YouTube
from unidecode import unidecode

In [55]:
#This function will helps us in the next function
#It compares two strings, takes the shortest, and the number of characters thats macthes on same indexes
def number_same_occ(a,b):

  #Initialization
  count = 0
  shortest = len(a) if len(a) <= len(b) else len(b)

  for i in range(shortest):
    if a[i].lower() == b[i].lower(): #if the two characters are the same at the i_th index
      count += 1
  return count

In [72]:
def add_corresponding_id(playlist_url,playlist_streamed):


  #The string list that will contains the ids. It will be added to playlist_url as a column
  ids = []

  #First, we fill playlist_url with the ids that are easily found through song name

  #we chech if the songs name in the 2 dataframes match. If it is the case, we add the song id in playlist_ms_url
  for i in range(len(playlist_url)):
    ids.append('')
    for j in range(len(playlist_streamed)):
      if str(playlist_url['songs'][i]).casefold() == str(playlist_streamed['track.name'][j]).casefold():
        ids[i] = playlist_streamed['track.id'][j]
        #We stop the loop  if we find a match
        break

  #We add the new column
  playlist_url['id'] = ids

  #The next treatment is in the case the i_th element doesn't have id and if the artists name appears once

  for i in range(len(playlist_url)):
    #if the ith element doesn't not have id, and there is only one occurence of the artist name

    a = str(playlist_url['Artistes'][i])
    try:
      if playlist_url['id'][i] == ''and playlist_streamed['track.artists'].str.casefold().value_counts()[a.casefold()] == 1:
        for j in range(len(playlist_streamed)):
          if str(playlist_url['Artistes'][i]).casefold() == str(playlist_streamed['track.artists'][j]).casefold():
            playlist_url['id'][i] = playlist_streamed['track.id'][j]
            break
    except KeyError:
      pass

  #The next treatment is for songs name left with no ids found yet

  for i in range(len(playlist_url)):

    #Initialization
    index = 10000
    count = 0

    #If the i_th element doesn't have id
    if playlist_url['id'][i] == '':
      for j in range(len(playlist_streamed)):
        if number_same_occ(playlist_url['songs'][i],playlist_streamed['track.name'][j]) > count: #We calculate the number of same occurrence on same index
          count = number_same_occ(playlist_url['songs'][i],playlist_streamed['track.name'][j]) #We update count value to the i_th element with the highest count value
          index = j

    #If nothing was found
    if index == 10000 and playlist_url['id'][i] == '':
      for k in range(len(playlist_streamed)):
        #CHeck if id does not exist in the url csv
        try:
          #This Error handling is just because value_counts can't return 0. It only returns values higher or equal to 1
          #In case nothing was found, it will be a KeyError

          temp = playlist_url['id'].str.casefold().value_counts()[playlist_streamed['track.id'][k].casefold()]
        except KeyError:
          print(playlist_url['Artistes'][i],playlist_streamed['track.artists'][k], number_same_occ(playlist_url['Artistes'][i],playlist_streamed['track.artists'][k]))
          if number_same_occ(playlist_url['Artistes'][i],playlist_streamed['track.artists'][k]) > count: #We calculate the number of same occurrence on same index
            count = number_same_occ(playlist_url['Artistes'][i],playlist_streamed['track.artists'][k]) #We update count value to the i_th element with the highest count value
            index = k

    #If id was found
    if index!= 10000 and j==len(playlist_streamed)-1:
        playlist_url['id'][i] = playlist_streamed['track.id'][index] #fill id of the i_th element

  return playlist_url

In [73]:
playlist_ms_url = pd.read_excel('/content/sample_data/most_streamed_url.xlsx')
playlist_most_streamed = pd.read_csv('/content/sample_data/most_streamed_playlist.csv')

In [74]:
playlist_ms_url = add_corresponding_id(playlist_ms_url,playlist_most_streamed)

In [75]:
for i in range(len(playlist_ms_url)):
  if playlist_ms_url['id'][i] == '':
    print(playlist_ms_url['songs'][i])
playlist_ms_url.head()

,Artistes,songs,url,id
0,YOASOBI,Idol,https://www.youtube.com/watch?v=ZRtdQ81jPUQ,1hAloWiinXLPQUJxrJReb1
1,Joe Hisaishi,Merry-Go-Round of Life,https://www.youtube.com/watch?v=2pQKqQ9sG50,39uLYYZytVUwcjgeYLI409
2,Fujii Kaze,Shinunoga E-Wa,https://www.youtube.com/watch?v=dawrQnvwMTY,0o9zmvc5f3EFApU52PPIyW
3,Creepy Nuts,Bling-Bang-Bang-Born,https://www.youtube.com/watch?v=mLW35YMzELE,0kdqcbwei4MDWFEX5f33yG
4,OFFICIAL HIGE DANDISM,Pretender,https://www.youtube.com/watch?v=MbhyzQiRFXY,15HNdxGKNCIO9pgaY4n7FU


In [76]:
playlist_ls_url = pd.read_excel('/content/sample_data/least_streamed_url.xlsx')
playlist_least_streamed = pd.read_csv('/content/sample_data/least_streamed_playlist.csv')

In [77]:
playlist_ls_url = add_corresponding_id(playlist_ls_url,playlist_least_streamed)

In [78]:
playlist_ls_url.head()

,Artistes,songs,url,id
0,The weeknd,Gasoline,https://www.youtube.com/watch?v=iOa7uE23qc4,3KyKxJ4P3pVCgaZwaq2rUC
1,The weeknd,Wanderlust,https://www.youtube.com/watch?v=vlrC-y1I3go,5OmX5lXxZ5sQgPuSHDn293
2,Fall Out Boy,Grenade Jumper,https://www.youtube.com/watch?v=oPIuxW2VIRE,3jsNPTt6LUyB2srOcZaq47
3,My Chemical Romance,This is the best day ever,https://www.youtube.com/watch?v=0zY0Ax6bijI,1mWMGKUaL7EyYIIveqlKC0
4,Panic! At the Disco,Folkin' Around,https://www.youtube.com/watch?v=PcgNKnVOPyc,58zSuvHCLIipfXWSKpdlSk


In [79]:
for i in range(len(playlist_ls_url)):
  if playlist_ls_url['id'][i] == '':
    print(playlist_ls_url['songs'][i])

In [80]:
#This function will helps in the function function
#It takes a list with all the files in a path and a youtube video title and see which file name matches better the title
#It returns the index of the file in the list

def check_name(all_files_name, yt_video_title):
  #We remove | character
  yt_video_title = unidecode(yt_video_title).lower()
  yt_video_title.replace('|',' ')

  #Remove 'Official video and music' from string
  yt_video_title = yt_video_title.replace('official','')
  yt_video_title = yt_video_title.replace('video','')
  yt_video_title = yt_video_title.replace('music','')

  #We split the string in a list
  title_word_list = yt_video_title.split(' ')
  #Just in case,we remove empty string in the list
  title_word_list = [x for x in title_word_list if x]

  #We initialize the variables
  count= 0
  temp_count = 0
  index = 100000

  #We check if there is only one word in the youtube title
  if len(title_word_list) == 1 and len(title_word_list[0]) > 3: #if only one word and if the word has a len > 3
    for j in range(len(all_files_name)):
      file_name = unidecode(all_files_name[j]).lower()#We remove accents and lower the case of word
      if title_word_list[0][:-4].lower() in file_name:
        return j

  #If the title has more than one word
  else:

    #The take each file in the file list
    for file_name in all_files_name:
      file_name_temp = unidecode(file_name).lower() #remove accent and lower the case
      for yt_word in title_word_list: #We check if each word in the youtube title macthes the file name
        if yt_word.lower() in file_name_temp:
          temp_count += 1
      if temp_count > count: #We update the index with the file name's index with the most matches with the youtube title words
        count = temp_count
        index = all_files_name.index(file_name)
      temp_count = 0
  return index

In [81]:
#This function will helps us find the path of the images that will be given to the Model
#We only need the name of files in spectrogram hit and non hit. We can iterate after for the spectrogram 2nd approach (hit or non hit)
#the same goes for mfcc, because the root name of images's file is the same

#So we can give the path of spectrogram (hit or non hit), or path of mfcc (hit or non hit)
def add_image_names(images_path, playlist_url):
  image = []

  #We create the file list through the images path
  path_spec = os.listdir(images_path)

  for i in range(len(playlist_url)):
    image.append('')
    #We remove ' characters in the youtube title
    try:
      file_name = YouTube(playlist_url['url'][i]).title.replace("'","")+'.png'
    except:
      #This is a special error handling because it apperas that one of the video is considered private (which is false since the video is public on youtube)
      file_name = 'Shakira - Te Dejo Madrid (Official HD Video)'
    #If title is found in the image list
    if file_name in path_spec or file_name[:-4] in path_spec:
      image[i] = path_spec[path_spec.index(file_name)]
      continue
    #If title not found, we do the check name function
    else:
        index = check_name(path_spec,file_name)
        if index == 100000:
          print(file_name, ' not found')
        else:
          image[i] = path_spec[index]

  return image

In [82]:
#We create image list for most streamed songs
images_ms = add_image_names('/content/drive/MyDrive/Projet hit non hit/mel-spectrograms hits',playlist_ms_url)

In [83]:
#We create image list for least streamed songs
images_ls = add_image_names('/content/drive/MyDrive/Projet hit non hit/mel-spectrograms non hits',playlist_ls_url)

In [84]:
#We add the new columns
playlist_ms_url['image_path'] = images_ms
playlist_ls_url['image_path'] = images_ls

In [85]:
#We create a column "boolean" column (streamed or non streamed)
playlist_ms_url['Streamed/Non-Streamed'] = np.ones(len(playlist_ms_url),dtype=int).tolist()
playlist_ls_url['Streamed/Non-Streamed'] = np.zeros(len(playlist_ls_url),dtype=int).tolist()

#We concatenate the two dataframes
playlist_total = pd.concat([playlist_ms_url,playlist_ls_url],ignore_index=True)

In [86]:
#We load the analysis and drop the irrelevant columns for both streamed and non streamed playlists
analysis_streamed = pd.read_csv('/content/sample_data/most_streamed_analysis.csv')
analysis_streamed = analysis_streamed.drop(['track_href','analysis_url','type','duration_ms','uri'], axis=1)

analysis_non_streamed = pd.read_csv('/content/sample_data/least_streamed_analysis.csv')
analysis_non_streamed = analysis_non_streamed.drop(['track_href','analysis_url','type','duration_ms','uri'], axis=1)

analysis_total = pd.concat([analysis_streamed,analysis_non_streamed],ignore_index=True)

In [87]:
#We join the analysis with the playlist_url dataframes with the help of the id
dataset_ms_ls = pd.merge(playlist_total, analysis_total, on='id', how='left')

In [88]:
print(dataset_ms_ls.shape)

(300, 18)


In [89]:
dataset_ms_ls = dataset_ms_ls.sample(frac=1).reset_index(drop=True)

In [90]:
print(analysis_total[analysis_total['id'].str.contains("0hPNfDQW0AHa1kaAknRanK")])

     danceability  energy  key  loudness  mode  speechiness  acousticness  \
219         0.428   0.904    5    -4.189     1        0.261        0.0404   

     instrumentalness  liveness  valence    tempo                      id  \
219          0.000004    0.0754    0.834  199.869  0hPNfDQW0AHa1kaAknRanK   

     time_signature  
219               4  


In [91]:
print(playlist_total[playlist_total['songs'].str.contains("All My Life")])

        Artistes        songs                                          url  \
223  Major Lazer  All My Life  https://www.youtube.com/watch?v=IW4KbVdISps   

                         id  \
223  0hPNfDQW0AHa1kaAknRanK   

                                            image_path  Streamed/Non-Streamed  
223  Major Lazer - All My Life (feat Burna Boy) (Of...                      0  


In [92]:
analysis_streamed.head(20)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,time_signature
0,0.457,0.941,10,-2.692,0,0.0917,0.10500,0.000001,0.4370,0.876,165.783,1hAloWiinXLPQUJxrJReb1,4
1,0.239,0.287,7,-12.565,0,0.0388,0.87600,0.644000,0.1220,0.185,71.608,39uLYYZytVUwcjgeYLI409,3
2,0.600,0.760,6,-6.124,0,0.0452,0.16600,0.000041,0.1890,0.519,158.078,0o9zmvc5f3EFApU52PPIyW,4
3,0.853,0.822,6,-6.836,0,0.2460,0.08410,0.009850,0.1660,0.746,157.013,0kdqcbwei4MDWFEX5f33yG,4
4,0.538,0.869,8,-3.464,1,0.0275,0.04700,0.000000,0.1400,0.369,91.972,15HNdxGKNCIO9pgaY4n7FU,4
5,0.625,0.668,6,-4.315,1,0.0249,0.28800,0.000000,0.1070,0.292,95.968,4IouQaO9GkaHC7AtMErdSa,4
6,0.727,0.937,11,-2.871,0,0.2860,0.00417,0.000000,0.0910,0.749,132.067,03UrZgTINDqvnUMbbIMhql,4
7,0.744,0.747,11,-4.281,0,0.0824,0.00691,0.000002,0.0739,0.368,120.077,4sgMbwl9lXGyHcHmtFYGMm,4
8,0.636,0.900,6,-3.568,1,0.0571,0.28600,0.000001,0.0982,0.686,134.004,37S86pw74OH8j96ZmMnrpR,4
9,0.806,0.707,7,-4.623,1,0.0307,0.33100,0.000000,0.2940,0.746,118.009,3d3ELsqKlQ7WA0a10Isu3l,4


In [93]:
dataset_ms_ls.head(20)

,Artistes,songs,url,id,image_path,Streamed/Non-Streamed,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,Natalie Imbruglia,On My Way,https://www.youtube.com/watch?v=hhnBDkgLJPg,0DEJ9GEuWuGFMR1GhPORnY,On My Way.png,0,0.633,0.690,7,-4.397,1,0.0298,0.02460,0.000005,0.1310,0.631,103.003,4
1,OFFICIAL HIGE DANDISM,Pretender,https://www.youtube.com/watch?v=MbhyzQiRFXY,15HNdxGKNCIO9pgaY4n7FU,Pretender (歌詞) official髭男dism.png,1,0.538,0.869,8,-3.464,1,0.0275,0.04700,0.000000,0.1400,0.369,91.972,4
2,Bryan Adams,Summer Of '69,https://www.youtube.com/watch?v=eFjjO_lhf9c,0GONea6G2XdnHWjNZd6zt3,Summer Of 69.png,1,0.509,0.834,2,-6.205,1,0.0386,0.01830,0.000000,0.0732,0.774,139.130,4
3,Fuse ODG,What it do,https://www.youtube.com/watch?v=gTyLD5gHAbg,2V3mQTPHYkxbkYxXMjnKFA,What It Do.png,0,0.834,0.878,7,-3.369,1,0.1200,0.00313,0.149000,0.2460,0.315,102.043,4
4,Édith Piaf,La vie en rose,https://www.youtube.com/watch?v=rzeLynj1GYM,4FmiciU3ZmfgABlbCSXcWw,La Vie en rose.png,1,0.390,0.302,8,-8.687,1,0.0321,0.98400,0.000000,0.0871,0.422,82.520,4
5,Bob Marley & The Wailers,Is This Love,https://www.youtube.com/watch?v=CHekNnySAfM,6JRLFiX9NJSoRRKxowlBYr,Bob Marley - Is This Love.png,1,0.776,0.561,6,-8.375,0,0.1030,0.11100,0.000000,0.0670,0.743,122.246,4
6,Jatin-Lalit,Chand Sifarish,https://www.youtube.com/watch?v=zWEOx7TSM6I,5EYZZvmNAH5VZCwuzYJqoA,Chand Sifarish Full Song Fanaa Aamir Khan K...,1,0.829,0.665,2,-8.128,1,0.0868,0.24200,0.000017,0.1230,0.861,122.043,4
7,Chris Brown,Heatbreak on a Full Moon,https://www.youtube.com/watch?v=FdTdOZOQ6V8,6bSLDFKAqEIItAGrdoygwY,Heartbreak on a Full Moon.png,0,0.613,0.576,4,-7.617,0,0.0651,0.00786,0.000005,0.1060,0.692,87.988,4
8,Grand Kalle,Indépendance Cha Cha,https://www.youtube.com/watch?v=RxkZ95PYcrM,3kVN8ze4Nfh3nfa8ZUxfpH,Grand Kalle - Independance Cha Champ4.png,1,0.888,0.742,4,-8.074,1,0.0306,0.77100,0.000699,0.0773,0.968,115.442,4
9,Najwa Karam,Laych Mgharrab,https://www.youtube.com/watch?v=_kJl5aWm9eg,12CSZVtObE3mSyVkCD3xvm,Najwa Karam - Laych Mgharrab [Official Audio] ...,0,0.439,0.606,0,-5.313,0,0.0410,0.32100,0.000000,0.3140,0.603,130.013,4


In [94]:
  dataset_ms_ls.to_excel('/content/sample_data/dataset_ms_ls.xlsx',index=False)